In [1]:
# General
from pprint import pprint
from collections import Counter
import os
import re
import logging
import string
import pickle
import numpy as np
import pandas as pd
import smart_open
import multiprocessing 
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

# Gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser

# NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer()

# Spacy
import spacy 

# Plotting
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

# Clustering 
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.manifold import TSNE

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


In [2]:
def clean_text(text):
    # Normalize tabs and remove newlines
    no_tabs = text.replace('\t', ' ').replace('\n', '').replace('Belt and Road','BRI');
    # Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs);
    # Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only);
    # Strip trailing and leading spaces
    no_spaces = multi_spaces.strip();
    return no_spaces

In [3]:
def sentence_tokenize(text):
    sentence_doc = sent_tokenize(text)
    sentences = [gensim.utils.simple_preprocess(str(doc), deacc=True) for doc in sentence_doc]  # deacc=True removes punctuations
    stop = set(stopwords.words('english') + ['’', '“', '”', 'nbsp', 'http','china','pakistan','nbbalu'])
    no_stop = [[word for word in sentence if word not in stop] for sentence in sentences]
    return no_stop

In [4]:
import os

path=os.listdir(r"C:\Users\Li\Desktop\《巴基斯坦时报》一带一路全部报道\txt")
datalist=[]

for i in path:
    domain= r"C:\Users\Li\Desktop\《巴基斯坦时报》一带一路全部报道\txt\\"+i
    #print(domain)
    with open(domain,'r',encoding='utf-8') as f:
        data=f.read()
        datalist.append(data)

text_li=[clean_text(i) for i in datalist]

In [5]:
text_li[0]

'China changes resonate with grassroot Pakistanis The Balochistan TimesNBBALU c The Balochistan Times Things had improved a lot here noted Asif Jalil a Pakistani businessman and restaurant owner based in Beijing China adding that marks the th anniversary of the establishment of diplomatic relations between China and Pakistan and he wishes China to emerge as a beautiful country.It was very lively before Jalil cherished the good old days since he had been here for more than years you could see restaurants had their tables outside with people sitting eating food enjoying beer chatting in the evening almost in every Hutong a narrow lane especially in summer. But this caused some pollution so slowly these things had gone. However he admitted that things and people had improved. Beijing has changed a lot he likes the atmosphere here a city renowned for its ancient opulent palaces temples and huge stone walls and gates. One thing is the physical change. There are more buildings and infrastruc

In [6]:
com_sent_li = [sentence_tokenize(text) for text in text_li]

In [8]:
com_sent_li[0][0]

['changes',
 'resonate',
 'grassroot',
 'pakistanis',
 'balochistan',
 'timesnbbalu',
 'balochistan',
 'times',
 'things',
 'improved',
 'lot',
 'noted',
 'asif',
 'jalil',
 'pakistani',
 'businessman',
 'restaurant',
 'owner',
 'based',
 'beijing',
 'adding',
 'marks',
 'th',
 'anniversary',
 'establishment',
 'diplomatic',
 'relations',
 'wishes',
 'emerge',
 'beautiful',
 'country',
 'lively',
 'jalil',
 'cherished',
 'good',
 'old',
 'days',
 'since',
 'years',
 'could',
 'see',
 'restaurants',
 'tables',
 'outside',
 'people',
 'sitting',
 'eating',
 'food',
 'enjoying',
 'beer',
 'chatting',
 'evening',
 'almost',
 'every',
 'hutong',
 'narrow',
 'lane',
 'especially',
 'summer']

In [9]:
sent_li = []
for sentence in com_sent_li:
    for tokens in sentence:
        sent_li.append(tokens)

In [10]:
sent_li = [tokens for sentence in com_sent_li for tokens in sentence]
sent_li

[['changes',
  'resonate',
  'grassroot',
  'pakistanis',
  'balochistan',
  'timesnbbalu',
  'balochistan',
  'times',
  'things',
  'improved',
  'lot',
  'noted',
  'asif',
  'jalil',
  'pakistani',
  'businessman',
  'restaurant',
  'owner',
  'based',
  'beijing',
  'adding',
  'marks',
  'th',
  'anniversary',
  'establishment',
  'diplomatic',
  'relations',
  'wishes',
  'emerge',
  'beautiful',
  'country',
  'lively',
  'jalil',
  'cherished',
  'good',
  'old',
  'days',
  'since',
  'years',
  'could',
  'see',
  'restaurants',
  'tables',
  'outside',
  'people',
  'sitting',
  'eating',
  'food',
  'enjoying',
  'beer',
  'chatting',
  'evening',
  'almost',
  'every',
  'hutong',
  'narrow',
  'lane',
  'especially',
  'summer'],
 ['caused', 'pollution', 'slowly', 'things', 'gone'],
 ['however', 'admitted', 'things', 'people', 'improved'],
 ['beijing',
  'changed',
  'lot',
  'likes',
  'atmosphere',
  'city',
  'renowned',
  'ancient',
  'opulent',
  'palaces',
  'templ

In [11]:
bigram = Phrases(sent_li, min_count=5, threshold=80)
trigram = Phrases(bigram[sent_li], threshold=80)  
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [12]:
trigrams = [trigram_mod[bigram_mod[sentence]] for sentence in sent_li]

In [13]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [103]:
num_features = 500        # Word vector dimensionality (how many features each word will be given)
min_word_count = 2        # Minimum word count to be taken into account
num_workers = cores       # Number of threads to run in parallel (equal to your amount of cores)
context = 7              # Context window size
downsampling = 0 #1e-2    # Downsample setting for frequent words
seed_n = 1                # Seed for the random number generator (to create reproducible results) 
sg_n = 1                  # Skip-gram = 1, CBOW = 0

model = Word2Vec(trigrams, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=seed_n, sg=sg_n)

In [104]:
from gensim.models import KeyedVectors

model.save("word2vec.vec")

model = Word2Vec.load("word2vec.vec")


In [107]:
print('{:,} terms in the vocabulary.'.format(len(model.wv)))
model.wv['free']

12,165 terms in the vocabulary.


array([ 2.37128045e-02,  1.54095560e-01,  4.66486625e-02, -7.88496360e-02,
       -2.99635306e-02, -7.58525729e-02,  8.52805078e-02,  2.04028323e-01,
        5.82013726e-02, -9.99904126e-02,  7.62931556e-02, -7.93947652e-02,
       -9.08396989e-02, -1.62944868e-02,  4.03667521e-03, -4.16824743e-02,
       -8.50794539e-02,  6.21009879e-02, -7.81974047e-02,  1.00325957e-01,
       -7.20391199e-02, -6.07100725e-02,  7.90200159e-02,  7.17517883e-02,
        4.01008986e-02,  8.45282227e-02,  8.29197466e-02, -5.20422533e-02,
       -1.17186502e-01, -1.70844477e-02, -8.13530907e-02, -2.15046722e-02,
        1.82138644e-02, -5.69444261e-02, -1.04360152e-02,  4.99947881e-03,
        5.53521030e-02,  4.85604554e-02, -3.86046134e-02, -1.02323875e-01,
       -7.87327737e-02,  1.48606673e-02, -1.46765947e-01,  2.12654155e-02,
       -1.48357183e-01,  3.29248980e-02, -5.25252409e-02,  6.33823723e-02,
        3.30044888e-02,  3.12875994e-02, -1.24336053e-02, -4.51720245e-02,
        5.10851704e-02, -

In [69]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token and print them as a formatted list
    """

    for word, similarity in model.wv.most_similar(positive=[token], topn=topn):
        print(word, round(similarity, 3))
        
get_related_terms('free')

low 0.91
taxation 0.908
abroad 0.903
foreigners 0.893
consumer 0.89
demands 0.889
preferential 0.888
advantageous 0.888
flexible 0.886
imported 0.885
incomes 0.884
zero_tariff 0.884
simplifying 0.881
relatively 0.879
profits 0.879
considerable 0.876
automobiles 0.874
excess 0.873
finish 0.873
cheaper 0.873


In [70]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = model.wv.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)
        
word_algebra(add=['real','free'])


spending


In [111]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;


In [113]:
Z = model.syn1neg

In [116]:
centers, clusters = clustering_on_wordvecs(Z, 10);
centroid_map = dict(zip(model.wv.index_to_key, clusters));

In [117]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
    # Use closest points for each cluster center to query closest 20 points to it
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
    # Query Word Index  for each position in the above array, and added to a Dictionary
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    # Create DataFrame from dictionary
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df

In [119]:
top_words = get_top_words(model.wv.index_to_key, 5000, centers, Z);

In [120]:
top_words[:10]

,Cluster #0,Cluster #1,Cluster #2,Cluster #3,Cluster #4,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9
1,dhaka,partially,renewables,undergraduate,peaceful_neighbourhood,manipulate,mushtaq,furthering,eve,absent
2,achakzai,kilowatts,substitution,decorated,consolidated,refute,hasnain,swiftly,mamnoon_hussain,mediate
3,id,oil_refinery,clusters,optional,reaffirming,dcpec,ishrat,thecountries,thanking,sake
4,chang,cable,incentive,winter,co_operative,theories,lt_general,theworld,presided,hostilities
5,mi,upgradation,synthetic,theatres,deepened,traps,ashfaq,witnessing,leg,spoilers
6,recitation,hectare,excess,findings,comprehensively,despitepakistan,haider,stellar,maiden,hatred
7,imam,seaport,comfortable,premises,elevate,jet,durrani,frontiers,jinpingand,principled
8,mary,acre,resultantly,shaanxi,carry_forward,reservoir,nadeem,embraces,boao_forum,urgency
9,afternoon,inland,locally,bachelors,attach,tend,desk,aligned,diaoyutai_state,predicament
10,nominated,phase_iii,revolutionizing,diploma,characterized,revolves,treasury,fits,lunch,determine
